In [ ]:
!pip install transformers datasets accelerate scikit-learn sentencepiece protobuf
!pip install torch

In [ ]:
import torch
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
import pandas as pd

print(" กำลังโหลดชุดข้อมูล Thai-True-Fake-News...")
dataset = load_dataset("EXt1/Thai-True-Fake-News")


df = pd.DataFrame(dataset['train'])


print("Available columns in df:", df.columns)

def map_label(status):
    if status == 'ข่าวปลอม': return 1
    return 0

df['text'] = df['Title'].fillna('')
df['label'] = df['Verification_Status'].apply(map_label)


df = df[['text', 'label']]


train_texts, val_texts, train_labels, val_labels = train_test_split(
    df['text'].tolist(), df['label'].tolist(), test_size=0.1, random_state=42
)


model_name = "airesearch/wangchanberta-base-att-spm-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)


def tokenize_function(texts):
    return tokenizer(texts, padding="max_length", truncation=True, max_length=400)


train_encodings = tokenize_function(train_texts)
val_encodings = tokenize_function(val_texts)

class ThaiDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self,):
        return len(self.labels)

train_dataset = ThaiDataset(train_encodings, train_labels)
val_dataset = ThaiDataset(val_encodings, val_labels)


model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    save_strategy="no",
    report_to="none"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset
)


print("เริ่มต้นการเทรนโมเดล... (ใช้เวลาประมาณ 10-20 นาที บน GPU)")
trainer.train()


save_path = "./sms_model"
model.save_pretrained(save_path)
tokenizer.save_pretrained(save_path)

print(f"เทรนเสร็จสิ้น! โมเดลถูกบันทึกไว้ที่โฟลเดอร์ {save_path}")

⏳ กำลังโหลดชุดข้อมูล Thai-True-Fake-News...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/747 [00:00<?, ?B/s]

dataset.csv: 0.00B [00:00, ?B/s]

Generating train split:   0%|          | 0/6004 [00:00<?, ? examples/s]

...

model.safetensors:   0%|          | 0.00/423M [00:00<?, ?B/s]

Some weights of CamembertForSequenceClassification were not initialized from the model checkpoint at airesearch/wangchanberta-base-att-spm-uncased and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


🚀 เริ่มต้นการเทรนโมเดล... (ใช้เวลาประมาณ 10-20 นาที บน GPU)


Step,Training Loss
10,0.692600
20,0.699800
30,0.702200
40,0.679600
50,0.663200
60,0.646000
70,0.684200
80,0.638200
90,0.616400
100,0.637100


✅ เทรนเสร็จสิ้น! โมเดลถูกบันทึกไว้ที่โฟลเดอร์ ./sms_model


In [ ]:
import shutil
from google.colab import files

print("กำลังบีบอัดไฟล์...")
shutil.make_archive('sms_model', 'zip', './sms_model')

print("⬇กำลังดาวน์โหลด sms_model.zip ...")
files.download('sms_model.zip')

📦 กำลังบีบอัดไฟล์...
⬇️ กำลังดาวน์โหลด sms_model.zip ...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>